# Pandas Pipelines

- The following notebook demonstrates how to create pandas pipelines by taking a small example of doing some basic data cleaning steps on the [Direct Marketing Amount Spent by Customers](https://www.kaggle.com/yoghurtpatil/direct-marketing) dataset available on kaggle datasets.


- We'll first do some operations like creating copy of the dataframe, dropping missing data, removing outliers, and converting columns to categorical datatype step-by-step.

- Then we'll replicate the same steps by creating a [Pandas pipeline](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pipe.html).

<hr>

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('data/DirectMarketing.csv')
df.tail().T

,995,996,997,998,999
Age,Young,Middle,Old,Middle,Young
Gender,Female,Male,Male,Male,Male
OwnHome,Rent,Rent,Own,Own,Rent
Married,Single,Single,Single,Married,Married
Location,Close,Far,Close,Close,Close
Salary,19400,40500,44800,79000,53600
Children,1,1,0,2,1
History,NaN,NaN,Medium,Medium,Medium
Catalogs,18,18,24,18,24
AmountSpent,384,1073,1417,671,973


In [2]:
copy_df = df.copy()

In [3]:
copy_df.isna().sum() / len(copy_df) * 100

Age             0.0
Gender          0.0
OwnHome         0.0
Married         0.0
Location        0.0
Salary          0.0
Children        0.0
History        30.3
Catalogs        0.0
AmountSpent     0.0
dtype: float64

In [4]:
copy_df.dropna(axis=1, thresh=len(df) * 0.7, inplace=True)

In [5]:
low = np.quantile(copy_df['Salary'], 0.05)
high = np.quantile(copy_df['Salary'], 0.95)

copy_df = copy_df[copy_df['Salary'].between(low, high)]

In [6]:
copy_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900 entries, 0 to 999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Age          900 non-null    object
 1   Gender       900 non-null    object
 2   OwnHome      900 non-null    object
 3   Married      900 non-null    object
 4   Location     900 non-null    object
 5   Salary       900 non-null    int64 
 6   Children     900 non-null    int64 
 7   Catalogs     900 non-null    int64 
 8   AmountSpent  900 non-null    int64 
dtypes: int64(4), object(5)
memory usage: 70.3+ KB


In [7]:
for col in copy_df.select_dtypes(include='object').columns:
    ratio = len(copy_df[col].value_counts()) / len(copy_df)
    print(f'{col}: {ratio}')
    copy_df[col] = copy_df[col].astype(
        'category') if ratio < 0.05 else copy_df[col]

Age: 0.0033333333333333335
Gender: 0.0022222222222222222
OwnHome: 0.0022222222222222222
Married: 0.0022222222222222222
Location: 0.0022222222222222222


In [8]:
copy_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900 entries, 0 to 999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Age          900 non-null    category
 1   Gender       900 non-null    category
 2   OwnHome      900 non-null    category
 3   Married      900 non-null    category
 4   Location     900 non-null    category
 5   Salary       900 non-null    int64   
 6   Children     900 non-null    int64   
 7   Catalogs     900 non-null    int64   
 8   AmountSpent  900 non-null    int64   
dtypes: category(5), int64(4)
memory usage: 40.0 KB


<hr>

- In above cells, we have created a copy of the original dataframe, dropped columns containing more than 30% missing data, removed outliers by only keeping between 5th and 95th percentile values of the **Salary** column, and finally converted objects datatype columns containing categories which are less than 5% of the entire dataset size.


- Now, we'll create separate functions for all of the above steps and use **Pandas.DataFrame.pipe** method to create a pandas pipeline.

<hr>

In [9]:
def make_copy(df):
    print('Making copy...')
    return df.copy()


def drop_missing(df):
    print('Dropping columns with more than 30% missing data...')
    return df.dropna(axis=1, thresh=len(df) * 0.7)


def remove_outliers(df, col_name):
    low = np.quantile(a=df[col_name], q=0.05)
    high = np.quantile(a=df[col_name], q=0.95)
    print('Removing outliers by keeping 5th and 95th percentile values...')
    return df[df[col_name].between(low, high, inclusive=True)]


def convert_to_categories(df):
    print(
        'Converting columns with less than 5% categories of the entire dataset size...'
    )
    for col in df.select_dtypes(include='object').columns:
        ratio = len(df[col].value_counts()) / len(df)
        df[col] = df[col].astype('category') if ratio < 0.05 else df[col]
    return df

In [10]:
df_cleaned = (
    df.pipe(
        make_copy
    ).pipe(
        drop_missing
    ).pipe(
        remove_outliers, 'Salary'
    ).pipe(
        convert_to_categories)
)

Making copy...
Dropping columns with more than 30% missing data...
Removing outliers by keeping 5th and 95th percentile values...
Converting columns with less than 5% categories of the entire dataset size...


In [11]:
df.tail()

,Age,Gender,OwnHome,Married,Location,Salary,Children,History,Catalogs,AmountSpent
995,Young,Female,Rent,Single,Close,19400,1,NaN,18,384
996,Middle,Male,Rent,Single,Far,40500,1,NaN,18,1073
997,Old,Male,Own,Single,Close,44800,0,Medium,24,1417
998,Middle,Male,Own,Married,Close,79000,2,Medium,18,671
999,Young,Male,Rent,Married,Close,53600,1,Medium,24,973


In [12]:
df_cleaned.tail()

,Age,Gender,OwnHome,Married,Location,Salary,Children,Catalogs,AmountSpent
995,Young,Female,Rent,Single,Close,19400,1,18,384
996,Middle,Male,Rent,Single,Far,40500,1,18,1073
997,Old,Male,Own,Single,Close,44800,0,24,1417
998,Middle,Male,Own,Married,Close,79000,2,18,671
999,Young,Male,Rent,Married,Close,53600,1,24,973


In [13]:
df.info(), df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Age          1000 non-null   object
 1   Gender       1000 non-null   object
 2   OwnHome      1000 non-null   object
 3   Married      1000 non-null   object
 4   Location     1000 non-null   object
 5   Salary       1000 non-null   int64 
 6   Children     1000 non-null   int64 
 7   History      697 non-null    object
 8   Catalogs     1000 non-null   int64 
 9   AmountSpent  1000 non-null   int64 
dtypes: int64(4), object(6)
memory usage: 78.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 900 entries, 0 to 999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Age          900 non-null    category
 1   Gender       900 non-null    category
 2   OwnHome      900 non-null    category
 3   Married      900 non-null  

(None, None)